# This notebook is for Coursera IBM Data science Capstone project!

In [11]:
import numpy as np
import pandas as pd

In [12]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [13]:
import requests
from bs4 import BeautifulSoup

In [14]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
web = requests.get(url)
web_url = web.text

In [15]:
soup = BeautifulSoup(web_url, 'lxml')

In [16]:
postal_table = soup.find('table', {'class':'wikitable sortable'})

In [17]:
header = []

for th in postal_table.tbody.find_all('th'):
    header.append(th.text.replace('\n', ''))

print('Header of dataframe: ', header)
df = pd.DataFrame(columns = header)

Header of dataframe:  ['Postal Code', 'Borough', 'Neighborhood']


In [18]:
postal_data = postal_table.tbody.find_all('tr')

In [19]:
for i in range(1, len(postal_data)):
    val = []

    for td in postal_data[i].find_all('td'):
        val.append(td.text.replace('\n', ''))
    
    df.loc[i] = val


In [20]:
print(df.shape)
print(df.head())

(180, 3)
  Postal Code           Borough               Neighborhood
1         M1A      Not assigned                           
2         M2A      Not assigned                           
3         M3A        North York                  Parkwoods
4         M4A        North York           Victoria Village
5         M5A  Downtown Toronto  Regent Park, Harbourfront


Remove the row where 'Borough' is 'Not assigned'

In [21]:
df = df[df['Borough'] != 'Not assigned']
print(df.head())

  Postal Code           Borough                                 Neighborhood
3         M3A        North York                                    Parkwoods
4         M4A        North York                             Victoria Village
5         M5A  Downtown Toronto                    Regent Park, Harbourfront
6         M6A        North York             Lawrence Manor, Lawrence Heights
7         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


If Borough exists and Neighborhood is Not assigned, then neighborhood is Borough

In [22]:
print(df[df['Neighborhood'] == 'Not assigned'])

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []


There is no any row with Borough value while Neighborhood is Not assigned.

In [23]:
print('Number of rows of my dataframe: ', df.shape)

Number of rows of my dataframe:  (103, 3)


# Now, let's add the latitude and longitude for locations


In [24]:
!pip install geocoder
import geocoder


     |████████████████████████████████| 102kB 7.4MB/s ta 0:00:011


# I'd found that using geocoder took too long time to fetch a latitude and longitude, so I decided to use the coordinates from CSV file directly.

In [30]:
!wget http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2020-05-28 01:04:52--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 161.202.50.39, 119.81.168.75, 119.81.168.76
Connecting to cocl.us (cocl.us)|161.202.50.39|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2020-05-28 01:04:53--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|161.202.50.39|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-28 01:04:55--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 103.116.4.197
Connecting to ibm.box.com (ibm.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1

In [33]:
geodata = pd.read_csv('Geospatial_Coordinates.csv')
print(geodata.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [38]:
print(df.info())
print(geodata.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 3 to 179
Data columns (total 3 columns):
Postal Code     103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB
None


# Join two dataframe together and show the Latitude and Longitude

In [39]:
combine = pd.merge(df, geodata, how = 'inner', on = 'Postal Code')
combine.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
